# REST Interact With `adk web`

Make sure you have started `adk web` locally.

In [33]:
import requests
from config import ADK_SESSION_CREATE_URL, ADK_RUN_SSE_URL, ADK_RUN_URL

In [34]:
ADK_SESSION_CREATE_URL, ADK_RUN_SSE_URL, ADK_RUN_URL

('http://localhost:8000/apps/document_agent/users/user/sessions',
 'http://localhost:8000/run_sse',
 'http://localhost:8000/run')

## Get A Session

In [44]:
# override for testing
ADK_SESSION_CREATE_URL = 'http://localhost:8000/apps'

In [45]:
agent_name = 'document_agent'
user_id = 'u_123'
session_id = 's_123'

In [ ]:
ADK_SESSION_URL = f"{ADK_SESSION_CREATE_URL}/{agent_name}/users/{user_id}/sessions/{session_id}"
AD

In [49]:
session_payload = {}
response = requests.post(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [],
 'lastUpdateTime': 1749120760.9617996}

In [50]:
SERVER_SESSION_ID = response_data['id']

## Interact With Session

In [55]:
message_parts = []
init_message = {'text': 'Hello'}
message_parts.append(init_message)

In [57]:
payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [58]:
response = requests.post(ADK_RUN_URL, json=payload, timeout=180)
response.raise_for_status()
response_data = response.json()
response_data

[{'content': {'parts': [{'text': 'Hi there! To get started, please upload a PDF or PNG file, or provide a GCS URI (bucket and path) for the document you want to process.\n'}],
   'role': 'model'},
  'usageMetadata': {'candidatesTokenCount': 35,
   'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 35}],
   'promptTokenCount': 3422,
   'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3422}],
   'totalTokenCount': 3457,
   'trafficType': 'ON_DEMAND'},
  'invocationId': 'e-4069d13a-8f13-46d6-9feb-ccdef3b17916',
  'author': 'document_agent',
  'actions': {'stateDelta': {},
   'artifactDelta': {},
   'requestedAuthConfigs': {}},
  'id': 'ppU9aYtf',
  'timestamp': 1749120962.253869}]

In [59]:
def parse_response(response_data):
    final_text = "Error: No final response from assistant found."
    if isinstance(response_data, list):
        for message in reversed(response_data):
            content = message.get('content', {})
            if isinstance(content, dict) and content.get("role") == "model":
                parts = content.get('parts', [])
                if parts and isinstance(parts, list):
                    first_part = parts[0]
                    if isinstance(first_part, dict):
                        text_part = first_part.get("text")
                        if text_part:
                            final_text = text_part
                        else:
                            final_text = f"Agent returned a structured response: `{json.dumps(first_part)}`"
                    else:
                        final_text = f"Agent returned an unknown part: `{str(first_part)}`"
                    break
    return final_text

parse_response(response_data)

'Hi there! To get started, please upload a PDF or PNG file, or provide a GCS URI (bucket and path) for the document you want to process.\n'

## Add To Interaction

In [61]:
message_parts = []
init_message = {'text': 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'}
message_parts.append(init_message)

payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [62]:
response = requests.post(ADK_RUN_URL, json=payload, timeout=180)
response.raise_for_status()
response_data = response.json()
response_data

[{'content': {'parts': [{'functionCall': {'id': 'adk-68715e9b-d953-4d24-ba39-b7a5c90c9127',
      'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf',
       'gcs_bucket': 'statmike-mlops-349915'},
      'name': 'get_gcs_file'}}],
   'role': 'model'},
  'usageMetadata': {'candidatesTokenCount': 56,
   'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 56}],
   'promptTokenCount': 3507,
   'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3507}],
   'totalTokenCount': 3563,
   'trafficType': 'ON_DEMAND'},
  'invocationId': 'e-c0d3361c-432e-4b8b-9ace-8711c5998738',
  'author': 'document_agent',
  'actions': {'stateDelta': {},
   'artifactDelta': {},
   'requestedAuthConfigs': {}},
  'longRunningToolIds': [],
  'id': 'x30PWwhF',
  'timestamp': 1749121149.59242},
 {'content': {'parts': [{'functionResponse': {'id': 'adk-68715e9b-d953-4d24-ba39-b7a5c90c9127',
      'name': 'get_gcs_file',
      're

In [63]:
parse_response(response_data)

'Okay, I have loaded the file from GCS. The artifact key is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`. What would you like to do next? I can perform extraction, classification, or comparison.\n'

## Fetch A Session

In [65]:
session_payload = {}
response = requests.get(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [{'content': {'parts': [{'text': 'Hello'}], 'role': 'user'},
   'invocationId': 'e-4069d13a-8f13-46d6-9feb-ccdef3b17916',
   'author': 'user',
   'actions': {'stateDelta': {},
    'artifactDelta': {},
    'requestedAuthConfigs': {}},
   'id': 'ZUAJbK53',
   'timestamp': 1749120962.252242},
  {'content': {'parts': [{'text': 'Hi there! To get started, please upload a PDF or PNG file, or provide a GCS URI (bucket and path) for the document you want to process.\n'}],
    'role': 'model'},
   'usageMetadata': {'candidatesTokenCount': 35,
    'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 35}],
    'promptTokenCount': 3422,
    'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3422}],
    'totalTokenCount': 3457,
    'trafficType': 'ON_DEMAND'},
   'invocationId': 'e-4069d13a-8f13-46d6-9feb-ccdef3b17916',
   'author': 'document_agent',
   'actions': {'stateDelta': {},
    'arti

---
### NOTES

- [ ] If artifact_delta then can i retrieve the artifact locally?
- [ ] How can i send a file as inline data - see app implementation for example
- [X] can i retrieve the entire session from the URL?